## 如果要爬蟲多支股票的資料
* 鴻海   https://tw.stock.yahoo.com/q/q?s=2317
* 台積電 https://tw.stock.yahoo.com/q/q?s=2330

In [4]:
from bs4 import BeautifulSoup
import requests
import time
import xlwings as xw

def yahoo_stock_crawler(stock_id):
    
    doc = requests.get(f"https://tw.stock.yahoo.com/q/q?s={stock_id}")
    html = BeautifulSoup(doc.text, 'html.parser')
    # 搜尋整個網頁裡，內容為 '個股資料' 的 html 標籤, 關聯到 table 最外層
    table = html.findAll(text="個股資料")[0].parent.parent.parent
    # 找尋 table 裡第二個 tr 標籤內所有的 td 標籤
    data_row = table.select("tr")[1].select("td")
    # 截取當日的交易價格(OHLC)，以及昨日收盤價
    last_close = data_row[7].text
    open_price = data_row[8].text
    high_price = data_row[9].text
    low_price  = data_row[10].text
    close_price = data_row[2].text
    
    return {
        "open": open_price,
        "high": high_price,
        "low" : low_price,
        "close": close_price,
        "lastClose": last_close
    }

In [5]:
# Test Foxconn
yahoo_stock_crawler(2317)

{'open': '82.5',
 'high': '83.4',
 'low': '82.3',
 'close': '82.5',
 'lastClose': '82.2'}

In [6]:
# Test TSMC
yahoo_stock_crawler(2330)

{'open': '329.5',
 'high': '331.0',
 'low': '329.0',
 'close': '329.5',
 'lastClose': '327.5'}

In [9]:
# 玉山 TSMC Foxconn 聯發科
stocks = [2884, 2330, 2317, 2454] 

for stock in stocks:
    print(stock, yahoo_stock_crawler(stock))

2884 {'open': '29.00', 'high': '29.15', 'low': '28.95', 'close': '28.95', 'lastClose': '28.85'}
2330 {'open': '329.5', 'high': '331.0', 'low': '329.0', 'close': '330.0', 'lastClose': '327.5'}
2317 {'open': '82.5', 'high': '83.4', 'low': '82.3', 'close': '82.5', 'lastClose': '82.2'}
2454 {'open': '381.0', 'high': '390.5', 'low': '380.0', 'close': '384.5', 'lastClose': '381.0'}


## 寫到excel裡

In [19]:
# 單元測試

wb = xw.Book(r"tw_stock_portfolio.xlsx")
wb.sheets # lists of all sheets
sheet = wb.sheets["TW2330"] # open sheet = "TW2330"
last_row = sheet.range("A1").end("down").row

stock_data = yahoo_stock_crawler(2330)

sheet.range(f"A{last_row+1}").value = time.strftime("%Y/%m/%d")
sheet.range(f"B{last_row+1}").value = stock_data['open']
sheet.range(f"C{last_row+1}").value = stock_data['high']
sheet.range(f"D{last_row+1}").value = stock_data['low']
sheet.range(f"E{last_row+1}").value = stock_data['close']
sheet.range(f"F{last_row+1}").value = stock_data['lastClose']

In [23]:
# 利用函數寫入所有的sheet

stocks = [2884, 2330, 2317] 

for stock in stocks:
    sheet = wb.sheets[f"TW{stock}"]
    print(sheet)
    
    last_row = sheet.range("A1").end("down").row
    print(last_row)
    
    stock_data = yahoo_stock_crawler(stock)
    print(stock_data)
    
    sheet.range(f"A{last_row+1}").value = time.strftime("%Y/%m/%d")
    sheet.range(f"B{last_row+1}").value = stock_data['open']
    sheet.range(f"C{last_row+1}").value = stock_data['high']
    sheet.range(f"D{last_row+1}").value = stock_data['low']
    sheet.range(f"E{last_row+1}").value = stock_data['close']
    sheet.range(f"F{last_row+1}").value = stock_data['lastClose']
    print("---")

<Sheet [tw_stock_portfolio.xlsx]TW2884>
26
{'open': '29.00', 'high': '29.15', 'low': '28.95', 'close': '29.05', 'lastClose': '28.85'}
---
<Sheet [tw_stock_portfolio.xlsx]TW2330>
26
{'open': '329.5', 'high': '331.0', 'low': '329.0', 'close': '329.5', 'lastClose': '327.5'}
---
<Sheet [tw_stock_portfolio.xlsx]TW2317>
26
{'open': '82.5', 'high': '83.4', 'low': '82.3', 'close': '82.5', 'lastClose': '82.2'}
---
